*아래 링크를 통해 이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.*

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/nlp-with-pytorch/blob/master/chapter_1/PyTorch_Basics.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/nlp-with-pytorch/blob/master/chapter_1/PyTorch_Basics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

# 샘플과 타깃의 인코딩

## TF 표현 (term frequency)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns

corpus = ['Time flies like an arrow.', 
          'Fruit flies like a banana.']
one_hot_vectorizer = CountVectorizer(binary=True) # tf

In [ ]:
sparse = one_hot_vectorizer.fit_transform(corpus)
print('sparse:\n', sparse) # sparse: 1의 위치만 인코딩해서 데이터에 쓰이는 메모리를 축소

In [ ]:
one_hot = sparse.toarray()
print('dense:\n', one_hot) # dense matrix (일반적인 행렬)

In [ ]:
vocab = one_hot_vectorizer.get_feature_names()
print('vocab:\n', vocab)

In [ ]:
# used 'seaborn' (sns) package
sns.heatmap(one_hot, annot=True,
            cbar=False, xticklabels=vocab,
            yticklabels=['Sentence 1', 'Sentence 2'])

# plt.savefig('1-04.png', dpi=300)
plt.show()

## TF-IDF 표현 (term frequency-inverse document frequency)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
 
tfidf_vectorizer = TfidfVectorizer() # tf-idf
tfidf = tfidf_vectorizer.fit_transform(corpus).toarray()
sns.heatmap(tfidf, annot=True, cbar=False, xticklabels=vocab,
            yticklabels= ['Sentence 1', 'Sentence 2'])

# plt.savefig('1-05.png', dpi=300)
plt.show()

$IDF(w) = \text{log} \left(\dfrac{N+1}{N_w+1}\right)+1$

첫 번째 문장의 'flies'와 'like'의 경우 TF = 1이므로 $\text{TF-IDF}=1\times\text{log}\left(\dfrac{2+1}{2+1}\right)+1=1$입니다.

단어 'an', 'arrow', 'time'의 경우 $N_w=1$입니다. 따라서 $\text{TF-IDF}=1\times\text{log}\left(\dfrac{2+1}{1+1}\right)+1=1.4054651081081644$입니다.

L2 정규화를 적용하면 'flies'와 'like'는 $\dfrac{1}{\sqrt{2\times1^2+3\times1.4054651081081644^2+}}=0.3552$가 됩니다.

'an', 'arrow', 'time'는 $\dfrac{1.4054651081081644}{\sqrt{2\times1^2+3\times1.4054651081081644^2+}}=0.4992$가 됩니다.

# 파이토치 기초

In [2]:
import torch
import numpy as np
torch.manual_seed(1234)

## 텐서

* 스칼라는 하나의 숫자입니다.
* 벡터는 숫자의 배열입니다.
* 행렬은 숫자의 2-D 배열입니다.
* 텐서는 숫자의 N-D 배열입니다.

#### 텐서 만들기

크기를 지정하여 텐서를 만들 수 있습니다. 여기서는 행이 5개이고 열이 3개인 텐서를 만듭니다.

In [7]:
def describe(x):
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))

In [8]:
describe(torch.Tensor(2, 3))

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[-1.6193e-24,  4.5909e-41, -1.6193e-24],
        [ 4.5909e-41,  0.0000e+00,  0.0000e+00]])


In [9]:
describe(torch.randn(2, 3))

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[ 0.0461,  0.4024, -1.0115],
        [ 0.2167, -0.6123,  0.5036]])


In [26]:
# Size print 
x.size()

torch.Size([2, 2])

In [27]:
x.shape

torch.Size([2, 2])

특정 크기의 랜덤한 텐서를 만드는 것이 일반적입니다.

In [10]:
x = torch.rand(2, 3)
describe(x)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.7749, 0.8208, 0.2793],
        [0.6817, 0.2837, 0.6567]])


1이나 0으로 채워진 텐서를 만들 수도 있습니다.

In [11]:
describe(torch.zeros(2, 3))
x = torch.ones(2, 3)
describe(x)

x.fill_(5)
describe(x)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0., 0., 0.],
        [0., 0., 0.]])
타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[1., 1., 1.],
        [1., 1., 1.]])
타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[5., 5., 5.],
        [5., 5., 5.]])


텐서를 초기화한 후 값을 바꿀 수 있습니다.

노트: 밑줄 문자(`_`)로 끝나는 연산은 인-플레이스 연산입니다.

In [12]:
x = torch.Tensor(3,4).fill_(5)
print(x.type())
print(x.shape)
print(x)

torch.FloatTensor
torch.Size([3, 4])
tensor([[5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.]])


리스트의 리스트로 텐서를 만들 수 있습니다.

In [13]:
x = torch.Tensor([[1, 2,],  
                  [2, 4,]])
describe(x)

타입: torch.FloatTensor
크기: torch.Size([2, 2])
값: 
tensor([[1., 2.],
        [2., 4.]])


넘파이 배열로 텐서를 만들 수 있습니다.

In [14]:
npy = np.random.rand(2, 3)
describe(torch.from_numpy(npy))
print(npy.dtype)

타입: torch.DoubleTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.7559, 0.3633, 0.4790],
        [0.1254, 0.9884, 0.4286]], dtype=torch.float64)
float64


#### 텐서 타입

The FloatTensor has been the default tensor that we have been creating all along

In [28]:
import torch
x = torch.arange(6).view(2, 3)
describe(x)

타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5]])


In [29]:
x = torch.FloatTensor([[1, 2, 3],  
                       [4, 5, 6]])
describe(x)

x = x.long()
describe(x)

x = torch.tensor([[1, 2, 3], 
                  [4, 5, 6]], dtype=torch.int64)
describe(x)

x = x.float() 
describe(x)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[1., 2., 3.],
        [4., 5., 6.]])
타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[1, 2, 3],
        [4, 5, 6]])
타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[1, 2, 3],
        [4, 5, 6]])
타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [30]:
x = torch.randn(2, 3)
describe(x)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[ 1.5385, -0.9757,  1.5769],
        [ 0.3840, -0.6039, -0.5240]])


In [31]:
describe(torch.add(x, x))

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[ 3.0771, -1.9515,  3.1539],
        [ 0.7680, -1.2077, -1.0479]])


In [32]:
describe(x + x)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[ 3.0771, -1.9515,  3.1539],
        [ 0.7680, -1.2077, -1.0479]])


In [35]:
x = torch.arange(6)
describe(x)

타입: torch.LongTensor
크기: torch.Size([6])
값: 
tensor([0, 1, 2, 3, 4, 5])


In [39]:
x = x.view(2, 3) # x.view()만으로는 x값이 변하지 않음
describe(x)

타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5]])


In [42]:
describe(torch.sum(x, dim=0)) # 행(0) 덧셈 (0+3, 1+4, 2+5)
describe(torch.sum(x, dim=1)) # 열(1) 덧셈 (0+1+2, 3+4+5)

타입: torch.LongTensor
크기: torch.Size([3])
값: 
tensor([3, 5, 7])
타입: torch.LongTensor
크기: torch.Size([2])
값: 
tensor([ 3, 12])


In [48]:
describe(x)
describe(torch.transpose(x, 0, 1)) # 행(0)과 열(1)을 전치시킴

타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5]])
타입: torch.LongTensor
크기: torch.Size([3, 2])
값: 
tensor([[0, 3],
        [1, 4],
        [2, 5]])
타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5]])


In [51]:
import torch
x_before_view = torch.arange(6)
describe(x_before_view)

타입: torch.LongTensor
크기: torch.Size([6])
값: 
tensor([0, 1, 2, 3, 4, 5])


In [53]:
x = x_before_view.view(2, 3)
describe(x)

타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5]])


In [56]:
# Indexing
describe(x[:1, :2])

타입: torch.LongTensor
크기: torch.Size([1, 2])
값: 
tensor([[0, 1]])


In [57]:
describe(x[0, 1])

타입: torch.LongTensor
크기: torch.Size([])
값: 
1


In [60]:
describe(x)
indices = torch.LongTensor([0, 2])
describe(torch.index_select(x, dim=1, index=indices))

타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5]])
타입: torch.LongTensor
크기: torch.Size([2, 2])
값: 
tensor([[0, 2],
        [3, 5]])


In [61]:
describe(x)
indices = torch.LongTensor([0, 0])
describe(torch.index_select(x, dim=0, index=indices))

타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5]])
타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [0, 1, 2]])


In [68]:
row_indices = torch.arange(2).long()
col_indices = torch.LongTensor([0, 1])
print('row_indices:', row_indices)
print('col_indices:', col_indices)
print('\n')

print('x:', x)
print('\n')

describe(x[row_indices, col_indices])

row_indices: tensor([0, 1])
col_indices: tensor([0, 1])


x: tensor([[0, 1, 2],
        [3, 4, 5]])


타입: torch.LongTensor
크기: torch.Size([2])
값: 
tensor([0, 4])


인덱싱 연산에는 넘파이 `int64` 타입에 해당하는 LongTensor가 사용됩니다.

In [69]:
x = torch.LongTensor([[1, 2, 3],  
                      [4, 5, 6],
                      [7, 8, 9]])
describe(x)
print(x.dtype)
print(x.numpy().dtype)

타입: torch.LongTensor
크기: torch.Size([3, 3])
값: 
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
torch.int64
int64


FloatTensor를 LongTensor로 바꿀 수 있습니다.

In [70]:
x = torch.FloatTensor([[1, 2, 3],  
                       [4, 5, 6],
                       [7, 8, 9]])
x = x.long()
describe(x)

타입: torch.LongTensor
크기: torch.Size([3, 3])
값: 
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


### 특별한 텐서 초기화

숫자가 증가되는 벡터를 만들 수 있습니다.

In [79]:
x = torch.arange(0., 10.)
describe(x)

타입: torch.FloatTensor
크기: torch.Size([10])
값: 
tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])


이따금 인덱싱을 위해 정수 기반의 배열이 필요합니다.

In [80]:
x = torch.arange(0., 10.).long()
describe(x)

타입: torch.LongTensor
크기: torch.Size([10])
값: 
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


## 연산

텐서로 선형 대수 계산을 하는 것은 최신 딥러닝 기술의 기초가 되었습니다.

파이토치의 `view` 메서드를 사용하면 원소의 순서를 유지하면서 텐서의 차원을 자유롭게 바꿀 수 있습니다.

In [81]:
x = torch.arange(0, 20)

print(x.view(1, 20))
print(x.view(2, 10))
print(x.view(4, 5))
print(x.view(5, 4))
print(x.view(10, 2))
print(x.view(20, 1))

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])
tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17],
        [18, 19]])
tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15],
        [16],
        [17],
        [18],
        [19]])


뷰를 사용하여 크기가 1인 차원을 추가할 수 있습니다. 이렇게 하면 다른 텐서와 연산할 때 브로드캐스팅을 활용할 수 있습니다.

In [82]:
x = torch.arange(12).view(3, 4)
y = torch.arange(4).view(1, 4)
z = torch.arange(3).view(3, 1)

print(x)
print(y)
print(z)

print(x + y)
print(x + z)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[0, 1, 2, 3]])
tensor([[0],
        [1],
        [2]])
tensor([[ 0,  2,  4,  6],
        [ 4,  6,  8, 10],
        [ 8, 10, 12, 14]])
tensor([[ 0,  1,  2,  3],
        [ 5,  6,  7,  8],
        [10, 11, 12, 13]])


`unsqueeze`와 `squeeze`는 크기가 1인 차원을 추가하고 삭제합니다.

In [87]:
x = torch.arange(12).view(3, 4)
print(x.shape)
describe(x)
print('\n')

x = x.unsqueeze(dim=1) # 1차원 추가
print(x.shape)
describe(x)
print('\n')

x = x.squeeze() # 1차원 삭제
print(x.shape)
describe(x)

torch.Size([3, 4])
타입: torch.LongTensor
크기: torch.Size([3, 4])
값: 
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


torch.Size([3, 1, 4])
타입: torch.LongTensor
크기: torch.Size([3, 1, 4])
값: 
tensor([[[ 0,  1,  2,  3]],

        [[ 4,  5,  6,  7]],

        [[ 8,  9, 10, 11]]])


torch.Size([3, 4])
타입: torch.LongTensor
크기: torch.Size([3, 4])
값: 
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


표준 수학 연산을 모두 지원합니다(예를 들어 `add`).

In [88]:
x = torch.rand(3,4)
print("x: \n", x)
print("--")
print("torch.add(x, x): \n", torch.add(x, x))
print("--")
print("x+x: \n", x + x)

x: 
 tensor([[0.6662, 0.3343, 0.7893, 0.3216],
        [0.5247, 0.6688, 0.8436, 0.4265],
        [0.9561, 0.0770, 0.4108, 0.0014]])
--
torch.add(x, x): 
 tensor([[1.3324, 0.6686, 1.5786, 0.6433],
        [1.0494, 1.3377, 1.6872, 0.8530],
        [1.9123, 0.1540, 0.8216, 0.0028]])
--
x+x: 
 tensor([[1.3324, 0.6686, 1.5786, 0.6433],
        [1.0494, 1.3377, 1.6872, 0.8530],
        [1.9123, 0.1540, 0.8216, 0.0028]])


메서드 이름 끝에 `_` 문자가 있으면 인-플레이스(in-place) 연산을 의미합니다.

In [89]:
x = torch.arange(12).reshape(3, 4)
print(x)
print(x.add_(x))

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[ 0,  2,  4,  6],
        [ 8, 10, 12, 14],
        [16, 18, 20, 22]])


차원을 줄이는 연산이 많이 있습니다. 예를 들면 `sum`입니다.

In [90]:
x = torch.arange(12).reshape(3, 4)
print("x: \n", x)
print("---")
print("행을 따라 덧셈 (dim=0): \n", x.sum(dim=0))
print("---")
print("열을 따라 덧셈 (dim=1): \n", x.sum(dim=1))

x: 
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
---
행을 따라 덧셈 (dim=0): 
 tensor([12, 15, 18, 21])
---
열을 따라 덧셈 (dim=1): 
 tensor([ 6, 22, 38])


#### 인덱싱, 슬라이싱, 연결, 수정

In [91]:
x = torch.arange(6).view(2, 3)
print("x: \n", x)
print("---")
print("x[:2, :2]: \n", x[:2, :2])
print("---")
print("x[0][1]: \n", x[0][1])
print("---")
print("[0][1]에 8을 할당")
x[0][1] = 8
print(x)

x: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
---
x[:2, :2]: 
 tensor([[0, 1],
        [3, 4]])
---
x[0][1]: 
 tensor(1)
---
[0][1]에 8을 할당
tensor([[0, 8, 2],
        [3, 4, 5]])


`index_select`을 사용해 텐서의 원소를 선택할 수 있습니다.

In [92]:
x = torch.arange(9).view(3,3)
print(x)

print("---")
indices = torch.LongTensor([0, 2])
print(torch.index_select(x, dim=0, index=indices))

print("---")
indices = torch.LongTensor([0, 2])
print(torch.index_select(x, dim=1, index=indices))

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
---
tensor([[0, 1, 2],
        [6, 7, 8]])
---
tensor([[0, 2],
        [3, 5],
        [6, 8]])


넘파이 스타일의 인덱싱도 사용할 수 있습니다.

In [95]:
x = torch.arange(9).view(3,3)
indices = torch.LongTensor([0, 2])
print('indices:', indices)
print("---")
print(x[indices])
print("---")
print(x[indices, :])
print("---")
print(x[:, indices])

indices: tensor([0, 2])
---
tensor([[0, 1, 2],
        [6, 7, 8]])
---
tensor([[0, 1, 2],
        [6, 7, 8]])
---
tensor([[0, 2],
        [3, 5],
        [6, 8]])


텐서를 연결할 수 있습니다. 먼저 행을 따라 열결합니다.

In [96]:
x = torch.arange(6).view(2,3)
describe(x)
describe(torch.cat([x, x], dim=0))
describe(torch.cat([x, x], dim=1))
describe(torch.stack([x, x]))

타입: torch.LongTensor
크기: torch.Size([2, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5]])
타입: torch.LongTensor
크기: torch.Size([4, 3])
값: 
tensor([[0, 1, 2],
        [3, 4, 5],
        [0, 1, 2],
        [3, 4, 5]])
타입: torch.LongTensor
크기: torch.Size([2, 6])
값: 
tensor([[0, 1, 2, 0, 1, 2],
        [3, 4, 5, 3, 4, 5]])
타입: torch.LongTensor
크기: torch.Size([2, 2, 3])
값: 
tensor([[[0, 1, 2],
         [3, 4, 5]],

        [[0, 1, 2],
         [3, 4, 5]]])


열을 따라 연결할 수 있습니다.

In [97]:
x = torch.arange(9).view(3,3)

print(x)
print("---")
new_x = torch.cat([x, x, x], dim=1)
print(new_x.shape)
print(new_x)

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
---
torch.Size([3, 9])
tensor([[0, 1, 2, 0, 1, 2, 0, 1, 2],
        [3, 4, 5, 3, 4, 5, 3, 4, 5],
        [6, 7, 8, 6, 7, 8, 6, 7, 8]])


텐서를 쌓아 새로운 0번째 차원에 연결할 수 있습니다.

In [ ]:
x = torch.arange(9).view(3,3)
print(x)
print("---")
new_x = torch.stack([x, x, x])
print(new_x.shape)
print(new_x)

#### 선형 대수 텐서 함수

전치는 다른 축의 차원을 서로 바꿉니다. 예를 들어 행과 열을 바꿀 수 있습니다.

In [ ]:
x = torch.arange(0, 12).view(3,4)
print("x: \n", x) 
print("---")
print("x.tranpose(1, 0): \n", x.transpose(1, 0))

3차원 텐서는 시퀀스의 배치로 표현됩니다. 시퀀스에 있는 각 아이템은 하나의 특성 벡터를 가집니다. 시퀀스 모델에서 시퀀스를 쉽게 인덱싱하기 위해 배치 차원과 시퀀스 차원을 바꾸는 일이 종종 있습니다.

노트: 전치는 2개의 축을 바꿉니다. `permute`는 여러 축을 다룰 수 있습니다(다음 셀에서 설명합니다).

In [ ]:
### [TODO] 여기서부터 다시!

In [98]:
batch_size = 3
seq_size = 4
feature_size = 5

x = torch.arange(batch_size * seq_size * feature_size).view(batch_size, seq_size, feature_size)

print("x.shape: \n", x.shape)
print("x: \n", x)
print("-----")

print("x.transpose(1, 0).shape: \n", x.transpose(1, 0).shape)
print("x.transpose(1, 0): \n", x.transpose(1, 0))

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
---
torch.Size([3, 3, 3])
tensor([[[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]],

        [[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]],

        [[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]]])


#### 선형 대수 텐서 함수

전치는 다른 축의 차원을 서로 바꿉니다. 예를 들어 행과 열을 바꿀 수 있습니다.

In [99]:
x = torch.arange(0, 12).view(3,4)
print("x: \n", x) 
print("---")
print("x.tranpose(1, 0): \n", x.transpose(1, 0))

x: 
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
---
x.tranpose(1, 0): 
 tensor([[ 0,  4,  8],
        [ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11]])


3차원 텐서는 시퀀스의 배치로 표현됩니다. 시퀀스에 있는 각 아이템은 하나의 특성 벡터를 가집니다. 시퀀스 모델에서 시퀀스를 쉽게 인덱싱하기 위해 배치 차원과 시퀀스 차원을 바꾸는 일이 종종 있습니다.

노트: 전치는 2개의 축을 바꿉니다. `permute`는 여러 축을 다룰 수 있습니다(다음 셀에서 설명합니다).

In [ ]:
### [TODO] 여기서부터 다시!

In [ ]:
batch_size = 3
seq_size = 4
feature_size = 5

x = torch.arange(batch_size * seq_size * feature_size).view(batch_size, seq_size, feature_size)

print("x.shape: \n", x.shape)
print("x: \n", x)
print("-----")

print("x.transpose(1, 0).shape: \n", x.transpose(1, 0).shape)
print("x.transpose(1, 0): \n", x.transpose(1, 0))

`permute`는 전치의 일반화된 버전입니다.

In [ ]:
batch_size = 3
seq_size = 4
feature_size = 5

x = torch.arange(batch_size * seq_size * feature_size).view(batch_size, seq_size, feature_size)

print("x.shape: \n", x.shape)
print("x: \n", x)
print("-----")

print("x.permute(1, 0, 2).shape: \n", x.permute(1, 0, 2).shape)
print("x.permute(1, 0, 2): \n", x.permute(1, 0, 2))

행렬 곱셈은 `mm`입니다.

In [ ]:
torch.randn(2, 3, requires_grad=True)

In [ ]:
x1 = torch.arange(6).view(2, 3).float()
describe(x1)

x2 = torch.ones(3, 2)
x2[:, 1] += 1
describe(x2)

describe(torch.mm(x1, x2))

In [ ]:
x = torch.arange(0, 12).view(3,4).float()
print(x)

x2 = torch.ones(4, 2)
x2[:, 1] += 1
print(x2)

print(x.mm(x2))

더 자세한 내용은 [파이토치 수학 연산 문서](https://pytorch.org/docs/stable/torch.html#math-operations)를 참고하세요!

## 그레이디언트 계산

In [ ]:
x = torch.tensor([[2.0, 3.0]], requires_grad=True)
z = 3 * x
print(z)

아래 간단한 코드에서 그레이디언트 계산을 엿볼 수 있습니다. 텐서 하나를 만들고 3을 곱합니다. 그다음 `sum()`을 사용해 스칼라 출력을 만듭니다. 손실 함수에는 스칼라 값이 필요하기 때문입니다. 그다음 손실에 `backward()`를 호출해 입력에 대한 변화율을 계산합니다. `sum()`으로 스칼라 값을 만들었기 때문에 `z`와 `x`에 있는 각 원소는 손실 스칼라 값에 대해 독립적입니다.

출력에 대한 `x`의 변화율은 `x`에 곱한 상수 3입니다.

In [ ]:
x = torch.tensor([[2.0, 3.0]], requires_grad=True)
print("x: \n", x)
print("---")
z = 3 * x
print("z = 3*x: \n", z)
print("---")

loss = z.sum()
print("loss = z.sum(): \n", loss)
print("---")

loss.backward()

print("loss.backward()를 호출한 후, x.grad: \n", x.grad)


### 예제: 조건 그레이디언트 계산하기

$$ \text{x=1에서 f(x)의 그레이디언트 찾기} $$
$$ {} $$
$$ f(x)=\left\{
\begin{array}{ll}
    sin(x) \; x>0 \text{ 일 때 }\\
    cos(x) \text{ 그 외 } \\
\end{array}
\right.$$

In [ ]:
def f(x):
    if (x.data > 0).all():
        return torch.sin(x)
    else:
        return torch.cos(x)

In [ ]:
x = torch.tensor([1.0], requires_grad=True)
y = f(x)
y.backward()
print(x.grad)

큰 벡터에 적용할 수 있지만 출력은 스칼라 값이어야 합니다.

In [ ]:
x = torch.tensor([1.0, 0.5], requires_grad=True)
y = f(x)
# 에러가 발생합니다!
y.backward()
print(x.grad)

스칼라 출력을 만들어 보죠.

In [ ]:
x = torch.tensor([1.0, 0.5], requires_grad=True)
y = f(x)
y.sum().backward()
print(x.grad)

하지만 이슈가 있습니다. 이 함수는 예외적인 경우에 맞지 않습니다.

In [ ]:
x = torch.tensor([1.0, -1], requires_grad=True)
y = f(x)
y.sum().backward()
print(x.grad)

In [ ]:
x = torch.tensor([-0.5, -1], requires_grad=True)
y = f(x)
y.sum().backward()
print(x.grad)

이는 원소별로 불리언 연산과 코사인/사인 계산이 수행되지 않기 때문입니다. 이를 해결하기 위해 자주 사용되는 방법은 마스킹입니다.

In [ ]:
def f2(x):
    mask = torch.gt(x, 0).float()
    return mask * torch.sin(x) + (1 - mask) * torch.cos(x)

x = torch.tensor([1.0, -1], requires_grad=True)
y = f2(x)
y.sum().backward()
print(x.grad)

In [ ]:
def describe_grad(x):
    if x.grad is None:
        print("그레이디언트 정보 없음")
    else:
        print("그레이디언트: \n{}".format(x.grad))
        print("그레이디언트 함수: {}".format(x.grad_fn))

In [ ]:
import torch
x = torch.ones(2, 2, requires_grad=True)
describe(x)
describe_grad(x)
print("--------")

y = (x + 2) * (x + 5) + 3
describe(y)
z = y.mean()
describe(z)
describe_grad(x)
print("--------")
z.backward(create_graph=True, retain_graph=True)
describe_grad(x)
print("--------")


In [ ]:
x = torch.ones(2, 2, requires_grad=True)

In [ ]:
y = x + 2

In [ ]:
y.grad_fn

### CUDA 텐서

파이토치 연산은 GPU나 CPU에서 수행할 수 있습니다. 두 장치를 사용하기 위한 몇 가지 연산을 제공합니다. (코랩에서 실행할 경우 런타임 유형을 GPU로 바꾸세요)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
x = torch.rand(3,3)
describe(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
x = torch.rand(3, 3).to(device)
describe(x)
print(x.device)

In [ ]:
cpu_device = torch.device("cpu")

In [ ]:
# 에러 발생!
y = torch.rand(3, 3)
x + y

In [ ]:
y = y.to(cpu_device)
x = x.to(cpu_device)
x + y

In [ ]:
if torch.cuda.is_available(): # GPU가 있을 경우에
    a = torch.rand(3,3).to(device='cuda:0') #  CUDA 텐서
    print(a)
    
    b = torch.rand(3,3).cuda()
    print(b)

    print(a + b)

    a = a.cpu() # 에러 발생
    print(a + b)

### 연습문제

연습문제에 필요한 일부 연산은 이 노트북에 있지 않습니다. [파이토치 문서](https://pytorch.org/docs/)를 참고하세요!

(정답은 맨 아래 있습니다)

#### 문제 1

2D 텐서를 만들고 차원 0 위치에 크기가 1인 차원을 추가하세요.

#### 문제 2 

이전 텐서에 추가한 차원을 삭제하세요.

#### 문제 3

[3, 7) 범위를 갖는 5x3 크기의 랜덤한 텐서를 만드세요.

#### 문제 4

정규 분포(평균=0, 표준편차=1)를 사용해 텐서를 만드세요.

#### 문제 5

텐서 `torch.Tensor([1, 1, 1, 0, 1])`에서 0이 아닌 원소의 인덱스를 추출하세요.

#### 문제 6

(3,1) 크기가 인 랜덤한 텐서를 만들고 네 벌을 복사해 쌓으세요.

#### 문제 7

두 개의 2차원 행렬(`a=torch.rand(3,4,5)`, `b=torch.rand(3,5,4)`)의 배치 행렬 곱셈(batch matrix-matrix product)을 계산하세요.

#### 문제 8

3차원 행렬(`a=torch.rand(3,4,5)`)과 2차원 행렬(`b=torch.rand(5,4)`)의 배치 행렬 곱셈을 계산하세요.

정답은 아래에..

정답은 더 아래에..

#### 문제 1

2D 텐서를 만들고 차원 0 위치에 크기가 1인 차원을 추가하세요.

In [ ]:
a = torch.rand(3,3)
a = a.unsqueeze(0)
print(a)
print(a.shape)

#### 문제 2 

이전 텐서에 추가한 차원을 삭제하세요.

In [ ]:
a = a.squeeze(0)
print(a.shape)

#### 문제 3

[3, 7) 범위를 갖는 5x3 크기의 랜덤한 텐서를 만드세요.

In [ ]:
3 + torch.rand(5, 3) * 4

#### 문제 4

정규 분포(평균=0, 표준편차=1)를 사용해 텐서를 만드세요.

In [ ]:
a = torch.rand(3,3)
a.normal_(mean=0, std=1)

#### 문제 5

텐서 `torch.Tensor([1, 1, 1, 0, 1])`에서 0이 아닌 원소의 인덱스를 추출하세요.

In [ ]:
a = torch.Tensor([1, 1, 1, 0, 1])
torch.nonzero(a)

#### 문제 6

(3,1) 크기가 인 랜덤한 텐서를 만들고 네 벌을 복사해 쌓으세요.

In [ ]:
a = torch.rand(3,1)
a.expand(3,4)

#### 문제 7

두 개의 2차원 행렬(`a=torch.rand(3,4,5)`, `b=torch.rand(3,5,4)`)의 배치 행렬 곱셈(batch matrix-matrix product)을 계산하세요.

In [ ]:
a = torch.rand(3,4,5)
b = torch.rand(3,5,4)
torch.bmm(a, b)

#### 문제 8

3차원 행렬(`a=torch.rand(3,4,5)`)과 2차원 행렬(`b=torch.rand(5,4)`)의 배치 행렬 곱셈을 계산하세요.

In [ ]:
a = torch.rand(3,4,5)
b = torch.rand(5,4)
torch.bmm(a, b.unsqueeze(0).expand(a.size(0), *b.size()))